In [1]:
import torch
assert torch.cuda.is_available(), "⚠️ Enable GPU: Runtime → Change runtime type → GPU"
print("🚀 Using GPU:", torch.cuda.get_device_name(0))

🚀 Using GPU: Tesla T4


In [2]:
# 🛠 Install compatible libraries
!pip install --upgrade datasets "numpy<2.0"
!pip install -q transformers datasets torch torchvision torchaudio kagglehub accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 815.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 827.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.4 MB/s eta 0:00:00


In [1]:
import os, glob, zipfile
import torch
import pandas as pd
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, DataCollatorWithPadding
)
from datasets import Dataset
import kagglehub

In [2]:
# ✅ Step 1: Verify GPU availability
assert torch.cuda.is_available(), (
    "⚠️ No GPU detected! "
    "Go to Runtime → Change runtime type → set Hardware accelerator to GPU"
)
device = torch.device("cuda")
print("🚀 Training on GPU:", torch.cuda.get_device_name(0))  # e.g., Tesla T4


🚀 Training on GPU: Tesla T4


In [13]:
# ✅ Step 2: Download dataset
print("📥 Downloading dataset…")
download_path = kagglehub.dataset_download("niyarrbarman/symptom2disease")
print("Download path:", download_path)

# ✅ Step 3: Locate CSV
all_files = glob.glob(os.path.join(download_path, "**/*.*"), recursive=True)
csv_file = next((f for f in all_files if f.lower().endswith(".csv")), None)
if not csv_file:
    zip_file = next((f for f in all_files if f.endswith(".zip")), None)
    if zip_file:
        with zipfile.ZipFile(zip_file, 'r') as z:
            z.extractall(download_path)
        all_files = glob.glob(os.path.join(download_path, "**/*.*"), recursive=True)
        csv_file = next((f for f in all_files if f.lower().endswith(".csv")), None)
if not csv_file:
    raise RuntimeError(f"No CSV found in {download_path}. Files: {all_files}")

print("✅ CSV found:", csv_file)
df = pd.read_csv(csv_file)
print("✅ Loaded", len(df), "rows across", df['label'].nunique(), "disease classes")
print(df.head())

# ✅ Step 4: Label mapping
# ✅ Step 4: Label mapping (unchanged)
labels = sorted(df['label'].unique())
label2id = {l:i for i,l in enumerate(labels)}
id2label = {i:l for l,i in label2id.items()}
df['label_id'] = df['label'].map(label2id)

# 🆕 New disease sample
new_samples = [
    {
        "text": "jar and khoki jaaro",
        "label": "kaala jar"
    }
]

for sample in new_samples:
    lbl = sample["label"]
    if lbl not in label2id:
        new_id = max(label2id.values()) + 1
        label2id[lbl] = new_id
        id2label[new_id] = lbl

    new_row_df = pd.DataFrame([{
        "text": sample["text"],
        "label": sample["label"],
        "label_id": label2id[sample["label"]]
    }])
    df = pd.concat([df, new_row_df], ignore_index=True)

print("Updated rows:", len(df), "| disease classes:", df['label'].nunique())
# 🔧 FIX: Update labels list after adding new samples
labels = sorted(df['label'].unique())  # Recalculate labels list
print("Updated rows:", len(df), "| disease classes:", len(labels))
# ✅ Step 5: create HF dataset
hf = Dataset.from_pandas(
    df[['text','label_id']].rename(columns={'label_id':'labels'}),
    preserve_index=False
)


📥 Downloading dataset…
Download path: /kaggle/input/symptom2disease
✅ CSV found: /kaggle/input/symptom2disease/Symptom2Disease.csv
✅ Loaded 1200 rows across 24 disease classes
   Unnamed: 0      label                                               text
0           0  Psoriasis  I have been experiencing a skin rash on my arm...
1           1  Psoriasis  My skin has been peeling, especially on my kne...
2           2  Psoriasis  I have been experiencing joint pain in my fing...
3           3  Psoriasis  There is a silver like dusting on my skin, esp...
4           4  Psoriasis  My nails have small dents or pits in them, and...
Updated rows: 1201 | disease classes: 25
Updated rows: 1201 | disease classes: 25


In [14]:

# ✅ Step 6: Tokenization
model_name = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
tokenizer = AutoTokenizer.from_pretrained(model_name)
def tokenize_fn(ex): return tokenizer(ex["text"], truncation=True, padding="max_length", max_length=512)
hf = hf.map(tokenize_fn, batched=True)
hf.set_format(type="torch", columns=["input_ids","attention_mask","labels"])

# ✅ Step 7: Load model onto GPU
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=len(labels), id2label=id2label, label2id=label2id
)
model.to(device)

Map:   0%|          | 0/1201 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [16]:
# ✅ Step 8: Setup Trainer with GPU
assert torch.cuda.is_available(), (
    "⚠️ No GPU detected! "
    "Go to Runtime → Change runtime type → set Hardware accelerator to GPU"
)
device = torch.device("cuda")
print("🚀 Training on GPU:", torch.cuda.get_device_name(0))  # e.g., Tesla T4


🚀 Training on GPU: Tesla T4


In [28]:
training_args = TrainingArguments(
    output_dir="./pubmedbert-s2d",
    num_train_epochs=9,
    per_device_train_batch_size=8,
    warmup_steps=50,
    weight_decay=0.01,
    logging_steps=10,
    eval_strategy="no", # Changed from evaluation_strateg
    save_strategy="epoch",
    # evaluation_strategy="epoch",
    # load_best_model_at_end=True,
    fp16=torch.cuda.is_available(),  # Mixed precision
    seed=42
)

In [29]:
trainer = Trainer(
    model=model, args=training_args,
    train_dataset=hf,
    data_collator=DataCollatorWithPadding(tokenizer),
    tokenizer=tokenizer,
)

# ✅ Step 9: Train & evaluate
print("🏋️ Training started…")
trainer.train()
print("✅ Training completed")
# print("📊 Evaluation metrics:", trainer.evaluate())

/tmp/ipython-input-29-2383808452.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


🏋️ Training started…


Step,Training Loss
10,0.045200
20,0.011600
30,0.010300
40,0.009200
50,0.048900
60,0.007500
70,0.006700
80,0.014000
90,0.214900
100,0.069600


✅ Training completed


In [30]:

# ✅ Step 10: Inference helper
def predict(text:str):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    for k in inputs: inputs[k] = inputs[k].to(device)
    model.eval()
    with torch.no_grad():
        probs = torch.nn.functional.softmax(model(**inputs).logits, dim=-1)
        idx = int(probs.argmax().item())
    return id2label[idx], float(probs[0, idx])

# 🧪 Test inference
# print(predict("I have a red, itchy, scaly rash on my elbows."))
print(predict("jar and khoki jaaro"))


('kaala jar', 0.9499890804290771)


In [31]:
predict("I've had a high fever, particularly at night. It's been quite unpleasant. There is a little headache, as well as constipation and diarrhea. I don't feel like eating anything.")

('Typhoid', 0.9998500347137451)

In [32]:
predict("I'm feeling fatigued and have no energy. I can barely keep my eyes open during the day, and I've been feeling lethargic and unable to motivate myself.")

('Chicken pox', 0.9997732043266296)

In [ ]:
predict("Recently, my skin has been quite itchy, and I have a rash all over my body. My skin also has a few spots where the hue is altered and some lumps and knot-like pimples.")

In [26]:
print(predict("jar and khoki jaaro"))

('Jaundice', 0.11904729902744293)
